In [4]:
import cv2
import face_recognition
import os
import urllib.request
from sklearn import neighbors

url = "https://github.com/opencv/opencv/raw/master/data/haarcascades/haarcascade_frontalface_default.xml"
xml_file_path = "haarcascade_frontalface_default.xml"
urllib.request.urlretrieve(url, xml_file_path)

current_directory = os.getcwd()
files_in_directory = os.listdir(current_directory)
xml_files = [file for file in files_in_directory if file.endswith(".xml")]


In [6]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
known_faces_dir = "C:/Users/MY LAPTOP/Desktop/faces_data"        #the faces data file 

known_faces = []
known_names = []

for person_name in os.listdir(known_faces_dir):
    person_path = os.path.join(known_faces_dir, person_name)

    if os.path.isdir(person_path):
        for image_filename in os.listdir(person_path):
            image_path = os.path.join(person_path, image_filename)
            image = face_recognition.load_image_file(image_path)
            face_encoding = face_recognition.face_encodings(image)[0]

            known_faces.append(face_encoding)
            known_names.append(person_name)

knn_clf = neighbors.KNeighborsClassifier(n_neighbors=len(known_faces))
knn_clf.fit(known_faces, known_names)

KNeighborsClassifier(n_neighbors=30)

In [7]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)


    for (x, y, w, h) in faces:
       
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        face_region = gray[y:y+h, x:x+w]
        face_encoding = face_recognition.face_encodings(frame, [(y, x+w, y+h, x)])[0]
        face_distances = face_recognition.face_distance(known_faces, face_encoding)
 
        threshold = 0.6  
        min_distance_index = face_distances.argmin()
        
        if face_distances[min_distance_index] < threshold:
            name = known_names[min_distance_index]
            box_color = (0, 255, 0)  
            text_color = (255, 255, 255)  
        else:
            name = "Unknown face"
            box_color = (0, 0, 255)  
            text_color = (0, 0, 255)
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), box_color, 2)
        font = cv2.FONT_HERSHEY_DUPLEX
        text_size = cv2.getTextSize(name, font, 1, 2)[0]
        cv2.putText(frame, name, (x - text_size[0], y + h), font, 1, text_color, 2)

    cv2.imshow('Face Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-uzca7qz1\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
